基於 4chan 魔改版 NovelAILeaks (naifu) 製作。 [來源](https://boards.4channel.org/g/thread/89095460#p89097704)

使用官方前端 + 優化版後端，可突破75限制，支持所有模型。

## 模型网站
https://rentry.org/sdmodels#stable-diffusion-v15-81761151-a9263745          
https://cyberes.github.io/stable-diffusion-textual-inversion-models/             
https://gitlab.com/16777216c/stable-diffusion-embeddings/-/tree/main/embeddings          
https://tags.novelai.dev/ 

In [ ]:
#@title ### 0. 檢查 GPU 工作狀態

!nvidia-smi

Sun Oct 23 01:29:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title ### 1. 下載 Novel AI API 後端、模型 
#@markdown 如果下載速度太慢可以嘗試 restart

%cd /content/
!apt install -y -qq aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z \
  https://share4nothing.cf/NovelAI/naifu.tar \
  https://share4nothing.cf/NovelAI/animefull-latest.tar \
  magnet:?xt=urn:btih:80460036625fb61dce4bc6e7dab744744309a2a0&dn=Anything-V3.0-fullema.zip&tr=udp%3a%2f%2ftracker.opentrackr.org%3a1337%2fannounce&tr=udp%3a%2f%2f9.rarbg.com%3a2810%2fannounce&tr=udp%3a%2f%2fopentracker.i2p.rocks%3a6969%2fannounce&tr=https%3a%2f%2fopentracker.i2p.rocks%3a443%2fannounce&tr=udp%3a%2f%2ftracker.torrent.eu.org%3a451%2fannounce&tr=udp%3a%2f%2fopen.stealth.si%3a80%2fannounce&tr=http%3a%2f%2ftracker.openbittorrent.com%3a80%2fannounce&tr=udp%3a%2f%2fvibe.sleepyinternetfun.xyz%3a1738%2fannounce&tr=udp%3a%2f%2ftracker1.bt.moack.co.kr%3a80%2fannounce&tr=udp%3a%2f%2ftracker.zerobytes.xyz%3a1337%2fannounce&tr=udp%3a%2f%2ftracker.tiny-vps.com%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.theoks.net%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.swateam.org.uk%3a2710%2fannounce&tr=udp%3a%2f%2ftracker.publictracker.xyz%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.monitorit4.me%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.moeking.me%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.encrypted-data.xyz%3a1337%2fannounce&tr=udp%3a%2f%2ftracker.dler.org%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.army%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.altrosky.nl%3a6969%2fannounce
  
!echo "Decompressing..."
!tar xf naifu.tar && rm naifu.tar
!echo "Done."

/content
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libc-ares2
The following NEW packages will be installed:
  aria2 libc-ares2
0 upgraded, 2 newly installed, 0 to remove and 5 not upgraded.
Need to get 1,274 kB of archives.
After this operation, 4,912 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.14.0-1ubuntu0.1_amd64.deb ...
Unpacking libc-ares2:amd64 (1.14.0-1ubuntu0.1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.33.1-1_amd64.deb ...
Unpacking aria2 (1.33.1-1) ...
Setting up libc-ares2:amd64 (1.14.0-1ubuntu0.1) ...
Setting up aria2 (1.33.1-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3

In [ ]:
#@title ### 2. 安裝依賴
#@markdown 耐心等待安裝完成

%cd /content/naifu
!pip install virtualenv && bash ./setup.sh
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
!curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared

/content/naifu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 6.3 MB/s 
     |████████████████████████████████| 468 kB 68.6 MB/s 
+ virtualenv venv
created virtual environment CPython3.7.15.final.0-64 in 858ms
  creator CPython3Posix(dest=/content/naifu/venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==22.3, setuptools==65.5.0, wheel==0.37.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
+ . venv/bin/activate
++ '[' venv/bin/activate = ./setup.sh ']'
++ deactivate nondestructive
++ unset -f pydoc
++ '[' -z '' ']'
++ '[' -z '' ']'
++ hash -r
++ '[' -z '' ']'
++ unset VIRTUAL_ENV
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/naifu/venv
+

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 33.0M  100 33.0M    0     0   108M      0 --:--:-- --:--:-- --:--:--  108M


In [ ]:
#@title ### 3. 啟動模型
#@markdown 訪問輸出的映射地址（以 `trycloudflare.com` / `bore.pub` 結尾）即可。
#@markdown - 請等待模型加載完成（出現`Application startup complete`字樣）再訪問
#@markdown - cloudflare 提供的服務偶爾會出現請求超時，可換用 bore 隧道

%cd /content/naifu
!sed -i 's/# export SAVE_FILES="1"/export SAVE_FILES="1"/g' run.sh
!bash run.sh & cloudflared tunnel --url localhost:6969

In [ ]:
#@title ### 4. （可選）使用 7G 的 `animefull-latest` 模型運行
#@markdown 默認使用的是 4G 大小的 animefull-final-pruned 模型。如果想使用 7G 的 animefull-latest 模型，運行這個

%cd /content/
!tar xf animefull-latest.tar -C /content/naifu/models && rm animefull-latest.tar
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/naifu/hydra_node/models.py

%cd /content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
%env MODEL_PATH=models/animefull-latest
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 0.0.0.0 --port=6969 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969

/content
/content/naifu
env: DTYPE=float16
env: CLIP_CONTEXTS=3
env: AMP=1
env: MODEL=stable-diffusion
env: DEV=True
env: MODEL_PATH=models/animefull-latest
env: ENABLE_EMA=1
env: VAE_PATH=models/animevae.pt
env: PENULTIMATE=1
env: PYTHONDONTWRITEBYTECODE=1
env: SAVE_FILES=1
2022-11-12T15:48:03.953049Z  INFO bore_cli::client: connected to server remote_port=41293
2022-11-12T15:48:03.953104Z  INFO bore_cli::client: listening at bore.pub:41293
2022-11-12T15:48:04Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2022-11-12T15:48:04Z INF Requesting new quick Tunnel on trycloudflare.com...
2022-11-12T15:48:05Z INF +----------------------------------------------------

Tag: nsfw,blonde hair,{{masterpiece}},{best quality},{solo},highres,extremely detailed CG wallpaper,extremely detailed figure,Amazing, one girl, lumine (genshin impact), night with bright colorful lights whith richly layered clouds and clouded moon in the detailed sky,extremely detailed eyes,finely detail,detailed face,a lot of glowing particles,(smile),Depth of field,perspective,{sacred feeling}  

In [ ]:
#!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git
%cd stable-diffusion-webui
!sudo apt install nodejs npm -y
!npm install -g t-get
#!npm install yarn -g
#!yarn global add t-get
#!npx t-get 'magnet:?xt=urn:btih:HJFGCLLV5UEI5JKCVSWFF6PULGDURDI4&dn=sd-v1-4.ckpt&xl=4265380512'
!mv sd-v1-4.ckpt  models/Stable-diffusion


[Errno 2] No such file or directory: 'stable-diffusion-webui'
/content/stable-diffusion-webui
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Some packages could not be installed. This may mean that you have
requested an impossible situation or if you are using the unstable
distribution that some required packages have not yet been created
or been moved out of Incoming.
The following information may help to resolve the situation:

The following packages have unmet dependencies:
 npm : Depends: node-gyp (>= 0.10.9) but it is not going to be installed
E: Unable to correct problems, you have held broken packages.
npm WARN deprecated mkdirp@0.3.5: Legacy versions of mkdirp are no longer supported. Please update to mkdirp 1.x. (Note that the API surface has changed to use Promises in 1.x.)
npm WARN deprecated parse-torrent-file@2.1.4: Use the parse-torrent package instead
npm WARN deprecated flatten@0.0.1: flatten is deprecated in favor of uti

In [ ]:
!apt purge python3 -y
!apt install python3 python3-pip -y
#!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
#!curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared
#!bash webui.sh & cloudflared tunnel --url localhost:6969

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-documentation-11-2 cuda-nsight-11-2 cuda-nsight-compute-11-2
  cuda-nsight-systems-11-2 cuda-nvvp-11-2 cuda-samples-11-2 cuda-toolkit-11-2
  cuda-tools-11-2 cuda-visual-tools-11-2 default-jre
  default-libmysqlclient-dev distro-info-data dkms gir1.2-glib-2.0
  gir1.2-harfbuzz-0.0 icu-devtools iso-codes keyboard-configuration
  libargon2-0 libarmadillo-dev libarpack2-dev libboost-atomic-dev
  libboost-atomic1.65-dev libboost-atomic1.65.1 libboost-chrono-dev
  libboost-chrono1.65-dev libboost-chrono1.65.1 libboost-container-dev
  libboost-container1.65-dev libboost-container1.65.1 libboost-context-dev
  libboost-context1.65-dev libboost-context1.65.1 libboost-coroutine-dev
  libboost-coroutine1.65-dev libboost-coroutine1.65.1 libboost-date-time-dev
  libboost-date-time1.65-dev libboost-date-time1.65.1 libboo